In [164]:
import pandas as pd
df = pd.read_csv('SO.csv')
df.shape

(100000, 6)

In [167]:
df.columns

Index(['Unnamed: 0', 'date', 'parent_id', 'score', 'title', 'reply'], dtype='object')

In [168]:
df.describe()

,Unnamed: 0,parent_id,score
count,100000.000000,1.000000e+05,100000.000000
mean,49999.500000,4.332477e+07,3.115090
std,28867.657797,1.076145e+07,9.590634
min,0.000000,8.050000e+02,1.000000
25%,24999.750000,3.813724e+07,1.000000
50%,49999.500000,4.431855e+07,1.000000
75%,74999.250000,5.097837e+07,3.000000
max,99999.000000,5.912274e+07,720.000000


In [ ]:
# Drop questions and answers w/ scores less than 1.
q_df = q_df[q_df.Score > 0]
a_df = a_df[a_df.Score > 0]

def no_code(text):
  '''Cleans code and html tags.'''
  text = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
  text = re.sub('<code>.*?</code>', '', text, flags=re.DOTALL)
  text = re.sub('<[^>]+>', '', text, flags=re.DOTALL)
  return text.replace("\n", "")

# Get rid of code and html tags from questions and answers.
q_df['Body'] = q_df.Body.apply(no_code)
a_df['Body'] = a_df.Body.apply(no_code)

# Drop questions and answers longer than 512 characters.
q_df = q_df.loc[q_df['Title'].str.len() < 511]
a_df = a_df.loc[a_df['Body'].str.len() < 511]

# Drop the answers that don't have a parent question.
a_df = a_df.loc[a_df.ParentId.isin(q_df.Id)]

q_df.shape, a_df.shape

In [ ]:
def clean_series(text):
    '''Clean html, new_line, nbsp from text'''

    # Series to return to work with pandas dfs easily
    new = pd.Series()
    
    # Things to replace
    html = re.compile('<.*?>')
    next_line = re.compile('\n')
    nbsp = re.compile('&nbsp')
    back_r = re.compile('\r')
    amp = re.compile('&amp')
    gt = re.compile('&gt')
    lt = re.compile('&lt')
    
    # Replacement steps
    new = re.sub(html, '', text)
    new = re.sub(next_line, ' ', new)
    new = re.sub(nbsp, ' ', new)
    new = re.sub(back_r, '', new)
    new = re.sub(amp, '', new)
    new = re.sub(gt, '', new)
    new = re.sub(lt, '', new)

    # Return cleaned series
    return new